# Liquidity Spike Tool — Demo

Demonstrates the `LiquiditySpikeTool` which computes four liquidity
metrics from local JSONL snapshot data:
1. **mean_liquidity** — average of open_interest (or volume)
2. **std_liquidity** — standard deviation
3. **latest_vs_mean_ratio** — latest / mean
4. **zscore_latest** — (latest − mean) / std

In [ ]:
import sys, json
from pathlib import Path
from datetime import datetime, timezone

PROJECT_ROOT = Path.cwd().parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

PREDICTION_AGENT_DIR = PROJECT_ROOT / "prediction_agent"
JSONL_PATH = PREDICTION_AGENT_DIR / "outputs" / "market_snapshots.jsonl"
print(f"JSONL: {JSONL_PATH}")

## 1. Load dataset and pick a market

In [ ]:
rows = []
with open(JSONL_PATH) as f:
    for line in f:
        line = line.strip()
        if line:
            rows.append(json.loads(line))

from collections import Counter
counts = Counter(r["market_id"] for r in rows)
target = counts.most_common(1)[0][0]
print(f"Total rows: {len(rows)}")
print(f"Selected market: {target} ({counts[target]} rows)")

## 2. Run the tool

In [ ]:
from prediction_agent.schemas import EventInput
from prediction_agent.tools.liquidity_spike_tool import LiquiditySpikeTool

event = EventInput(event_id="demo", market_id=target, market_title="Demo", current_price=0.50)
tool = LiquiditySpikeTool(jsonl_path=JSONL_PATH)
result = tool.run(event, window_minutes=999_999)

print("=== Tool Output ===")
print(f"tool_name    : {result.tool_name}")
print(f"output_vector: {result.output_vector}")
print(f"confidence   : {result.metadata['confidence']}")
print(f"sample_count : {result.metadata['sample_count']}")

labels = ["mean_liquidity", "std_liquidity", "latest_vs_mean_ratio", "zscore_latest"]
print("\nBreakdown:")
for label, val in zip(labels, result.output_vector):
    print(f"  {label:24s} = {val}")

## 3. Plot liquidity series over time

In [ ]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

market_rows = [r for r in rows if r["market_id"] == target]

timestamps, liq_vals = [], []
for r in market_rows:
    oi = r.get("open_interest")
    vol = r.get("volume")
    if oi is not None:
        val = oi
    elif vol is not None:
        val = vol
    else:
        continue
    timestamps.append(datetime.fromisoformat(r["timestamp"]))
    liq_vals.append(val)

pairs = sorted(zip(timestamps, liq_vals))
timestamps = [p[0] for p in pairs]
liq_vals = [p[1] for p in pairs]

fig, ax = plt.subplots(figsize=(12, 5))
ax.bar(timestamps, liq_vals, width=0.001, color="#16a34a", alpha=0.7, label="OI / Volume")
mean_liq = result.output_vector[0]
ax.axhline(y=mean_liq, color="gray", linestyle="--", alpha=0.6, label=f"mean={mean_liq:.1f}")
ax.set_title(f"Liquidity vs Time — {target}", fontsize=14)
ax.set_xlabel("Timestamp (UTC)")
ax.set_ylabel("Liquidity (OI or Volume)")
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
fig.autofmt_xdate()
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.savefig(str(PREDICTION_AGENT_DIR / "outputs" / "liquidity_vs_time.png"), dpi=150)
plt.show()
print("Plot saved.")